In [1]:
### RDKit environment ###

In [2]:
"""Select the compounds which have AE data from FAERS or SIDER so that we can do target prediction"""

'Select the compounds which have AE data from FAERS or SIDER so that we can do target prediction'

In [3]:
import os
import re
import sqlite3 as sqlite
import pandas as pd
import pymysql
from rdkit import Chem
from rdkit.Chem import inchi
import sys
from standardiser import break_bonds, unsalt, neutralise, rules, standardise
import logging

RDKit WARNING: [13:21:54] Enabling RDKit 2019.09.2 jupyter extensions


In [4]:
basedir = '/scratch/ias41/ae_code'

In [5]:
# AEOLUS ids for compounds for which there is an ae_associations file
filenames = [i for i in os.listdir(basedir + '/psm_aeolus/results/data') if ('ae_associations' in i and 'PSM' not in i)]
pattern = re.compile('([0-9]+)_*')
aeolus_ids = [pattern.match(filename).group(1) for filename in filenames]

In [6]:
# Find molregnos from database

conn = sqlite.connect(basedir + '/compound_mapping/results/201903_mapped_compounds_calculon.db')
cur = conn.cursor()
result = cur.execute("select distinct aeolus_concept, mapped_parent_molregno, mapped_parent_standard_inchi from compound_structures where mapped_parent_molregno is not null").fetchall()
conn.close()

In [7]:
# Make a dictionary of aeolus2inchi
aeolus2info = dict()

for item in result:
    aeolus2info[item[0]] = {'molregno': item[1], 'inchi': item[2]}

In [8]:
# Find molregnos for the aeolus_ids that have AE data
compound_info = []
errors = []
for aeolus_id in aeolus_ids: 
    try:
        info = aeolus2info[int(aeolus_id)]
        compound_info.append(info)
    except KeyError:
        errors.append(aeolus_id)

In [9]:
# This is the number of compounds that were mapped to ChEMBL ids and also had enough reports etc during the PSM process,
# e.g. 5 reports in each cell to derive AE-associations
len(compound_info)

2012

### Add sider compounds

In [10]:
sider_all_se = pd.read_csv(basedir + '/sider/results/sider_all_se_pt_mapped.txt', sep='\t')

In [11]:
sider_molregnos = set(sider_all_se['parent_molregno'].drop_duplicates())

In [12]:
mysql_defaults_file = '/scratch/ias41/ae_code/bioactivities/mysql_info_chembl.txt'

In [13]:
calculon_chembl = pymysql.connect(read_default_file=mysql_defaults_file, unix_socket='/var/run/mysqld/mysqld.sock')

In [14]:
def do_query(myquery):
    try:
        with calculon_chembl.cursor() as cursor:
            cursor.execute(myquery)
            result = cursor.fetchall()
            return(result)
    except:
        e = sys.exc_info()
        print(str(e))

In [15]:
sider_molregno_str = ', '.join([str(i) for i in sider_molregnos])
my_query = f"""select distinct molregno, standard_inchi from compound_structures where molregno in ({sider_molregno_str});"""

In [16]:
sider_result = do_query(my_query)

In [17]:
# Make a dictionary of molregno2info for sider compounds
sider_molregno2info = dict()

for item in sider_result:
    sider_molregno2info[item[0]] = {'molregno': item[0], 'inchi': item[1]}

In [18]:
# Unique molregno-inchi pairs for FAERS and SIDER compounds
all_compounds = set(((i['molregno'], i['inchi']) for i in sider_molregno2info.values())) | set((j['molregno'], j['inchi']) for j in aeolus2info.values())

In [19]:
len(all_compounds)

2546

### Retrieve canonical smiles from ChEMBL

In [20]:
combined_molregnos_str = ', '.join([str(i[0]) for i in all_compounds])

In [21]:
smiles_query = f"""select distinct molregno, canonical_smiles from compound_structures where molregno in ({combined_molregnos_str});"""

In [22]:
smiles_result = do_query(smiles_query)

In [23]:
smiles_dict = dict()
for item in smiles_result:
    smiles_dict[int(item[0])] = item[1]

In [24]:
compound_df = pd.DataFrame(all_compounds, columns = ['molregno', 'inchi'])

In [25]:
def find_smiles(x):
    try:
        smiles = smiles_dict[x]
        return smiles
    except KeyError:
        return None

In [26]:
compound_df['smiles'] = compound_df['molregno'].apply(lambda x: find_smiles(x))

In [27]:
len(compound_df)

2546

In [28]:
len(compound_df.loc[compound_df['smiles'].isnull()])

74

In [29]:
selection = compound_df.loc[~compound_df['smiles'].isnull(),['smiles', 'molregno']]

In [30]:
len(selection)

2472

### Try to standardise compounds

In [31]:
def try_to_standardise(smiles):
    """Return standardised smiles, excepting standardise errors returning None.
    mol -- RDKit mol object"""
    mol = Chem.MolFromSmiles(smiles)
    try:
        standardised_molecule = standardise.run(mol)
        standard_smiles = Chem.MolToSmiles(standardised_molecule)
        return standard_smiles
    
    except standardise.StandardiseException as e:
        logging.warn(e.message)
        return None    

In [32]:
selection['standard_smiles'] = selection['smiles'].apply(lambda x: try_to_standardise(x))

/scratch/ias41/envs/my-rdkit-env/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  # This is added back by InteractiveShellApp.init_path()
RDKit ERROR: [13:22:51] Can't kekulize mol.  Unkekulized atoms: 0 2 4 6 7 9
RDKit ERROR: 
RDKit ERROR: [13:22:52] Can't kekulize mol.  Unkekulized atoms: 3 10
RDKit ERROR: 


In [33]:
standard_selection = selection.loc[~selection['standard_smiles'].isnull(),['standard_smiles', 'molregno']]

In [34]:
standard_selection.to_csv(basedir + '/bioactivities/data/pidgin_input.smi', sep=' ', index=False, header=False)

In [37]:
standard_selection[:10].to_csv(basedir + '/bioactivities/data/pidgin_input_test.smi', sep=' ', index=False, header=False)

In [35]:
calculon_chembl.close()

In [36]:
standard_selection.head()

,standard_smiles,molregno
0,CC(S)C(=O)NCC(=O)O,264400
1,C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4...,428747
2,O=C1C2CC=CCC2C(=O)N1SC(Cl)(Cl)Cl,388389
3,CC(=O)OCC[N+](C)(C)C,27812
4,CC(=O)CCC(=O)O,698281
